# 06: BER Simulation

This notebook covers Monte Carlo Bit Error Rate (BER) simulation.

## Learning Objectives
- Understand BER as a performance metric
- Run Monte Carlo simulations
- Compare measured vs theoretical BER
- Analyze different modulation schemes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
from r4w_python import plot_ber_curve, plot_ber_comparison

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## BER Fundamentals

**Bit Error Rate (BER)** = Number of bit errors / Total bits transmitted

Typical requirements:
- Voice: BER < $10^{-3}$
- Data: BER < $10^{-6}$
- Financial: BER < $10^{-9}$

## Theoretical BER for AWGN

For common modulation schemes over AWGN:

| Modulation | BER Formula |
|------------|-------------|
| BPSK | $Q(\sqrt{2E_b/N_0})$ |
| QPSK | $Q(\sqrt{2E_b/N_0})$ |
| M-PSK | $\approx \frac{2}{\log_2 M} Q(\sqrt{2\log_2 M \cdot E_b/N_0} \sin(\pi/M))$ |

Where $Q(x) = \frac{1}{2}\text{erfc}(x/\sqrt{2})$

In [ ]:
def q_function(x):
    """Q function (tail probability of standard normal)."""
    return 0.5 * special.erfc(x / np.sqrt(2))

def ber_bpsk_theory(eb_n0_db):
    """Theoretical BPSK BER."""
    eb_n0 = 10**(eb_n0_db / 10)
    return q_function(np.sqrt(2 * eb_n0))

def ber_qpsk_theory(eb_n0_db):
    """Theoretical QPSK BER (same as BPSK for same Eb/N0)."""
    return ber_bpsk_theory(eb_n0_db)

def ber_8psk_theory(eb_n0_db):
    """Approximate theoretical 8-PSK BER."""
    eb_n0 = 10**(eb_n0_db / 10)
    return (2/3) * q_function(np.sqrt(2 * 3 * eb_n0) * np.sin(np.pi/8))

def ber_16qam_theory(eb_n0_db):
    """Approximate theoretical 16-QAM BER."""
    eb_n0 = 10**(eb_n0_db / 10)
    return (3/8) * special.erfc(np.sqrt((4/10) * eb_n0))

In [ ]:
# Plot theoretical BER curves
eb_n0_db = np.linspace(0, 15, 100)

plt.figure(figsize=(10, 6))
plt.semilogy(eb_n0_db, ber_bpsk_theory(eb_n0_db), label='BPSK')
plt.semilogy(eb_n0_db, ber_qpsk_theory(eb_n0_db), '--', label='QPSK')
plt.semilogy(eb_n0_db, ber_8psk_theory(eb_n0_db), label='8-PSK')
plt.semilogy(eb_n0_db, ber_16qam_theory(eb_n0_db), label='16-QAM')

plt.xlabel('Eb/N0 (dB)')
plt.ylabel('Bit Error Rate')
plt.title('Theoretical BER Curves (AWGN Channel)')
plt.legend()
plt.grid(True, which='both')
plt.ylim(1e-7, 1)
plt.show()

## Monte Carlo Simulation

Steps:
1. Generate random bits
2. Modulate to symbols
3. Add AWGN noise
4. Demodulate (hard decision)
5. Count bit errors
6. Repeat until statistically significant

In [ ]:
def simulate_bpsk(eb_n0_db, num_bits=100000):
    """Simulate BPSK over AWGN and return BER."""
    # Generate random bits
    bits = np.random.randint(0, 2, num_bits)
    
    # BPSK modulation: 0 -> -1, 1 -> +1
    symbols = 2 * bits - 1
    
    # Add AWGN
    # For BPSK: SNR = Eb/N0 (symbol energy = bit energy)
    eb_n0 = 10**(eb_n0_db / 10)
    noise_std = np.sqrt(1 / (2 * eb_n0))
    noise = noise_std * np.random.randn(num_bits)
    rx = symbols + noise
    
    # Hard decision demodulation
    detected_bits = (rx > 0).astype(int)
    
    # Count errors
    errors = np.sum(bits != detected_bits)
    return errors / num_bits

In [ ]:
# Run BPSK simulation
eb_n0_test = np.arange(0, 12, 1)
ber_simulated = []

for eb_n0 in eb_n0_test:
    # More bits for lower BER to get good statistics
    num_bits = min(1000000, max(100000, int(100 / ber_bpsk_theory(eb_n0) + 1)))
    ber = simulate_bpsk(eb_n0, num_bits)
    ber_simulated.append(ber)
    print(f"Eb/N0 = {eb_n0} dB: BER = {ber:.2e} ({num_bits} bits)")

In [ ]:
# Compare simulated vs theoretical
plt.figure(figsize=(10, 6))

# Theoretical
eb_n0_fine = np.linspace(0, 12, 100)
plt.semilogy(eb_n0_fine, ber_bpsk_theory(eb_n0_fine), 'b-', label='BPSK Theory')

# Simulated
ber_array = np.array(ber_simulated)
ber_array[ber_array == 0] = 1e-7  # Avoid log(0)
plt.semilogy(eb_n0_test, ber_array, 'ro', markersize=8, label='BPSK Simulated')

plt.xlabel('Eb/N0 (dB)')
plt.ylabel('Bit Error Rate')
plt.title('BPSK: Simulated vs Theoretical BER')
plt.legend()
plt.grid(True, which='both')
plt.ylim(1e-6, 1)
plt.show()

## QPSK Simulation

In [ ]:
def simulate_qpsk(eb_n0_db, num_bits=100000):
    """Simulate QPSK over AWGN and return BER."""
    # Ensure even number of bits
    num_bits = (num_bits // 2) * 2
    bits = np.random.randint(0, 2, num_bits)
    
    # Gray-coded QPSK constellation
    # 00 -> exp(j*pi/4), 01 -> exp(j*3pi/4), etc.
    symbols = np.zeros(num_bits // 2, dtype=complex)
    for i in range(num_bits // 2):
        b0, b1 = bits[2*i], bits[2*i + 1]
        # Gray code mapping
        I = 1 - 2*b0
        Q = 1 - 2*b1
        symbols[i] = (I + 1j*Q) / np.sqrt(2)
    
    # Add AWGN
    # For QPSK: Es/N0 = 2*Eb/N0 (2 bits per symbol)
    eb_n0 = 10**(eb_n0_db / 10)
    es_n0 = 2 * eb_n0
    noise_std = np.sqrt(1 / (2 * es_n0))
    noise = noise_std * (np.random.randn(len(symbols)) + 1j * np.random.randn(len(symbols)))
    rx = symbols + noise
    
    # Hard decision demodulation
    detected_bits = np.zeros(num_bits, dtype=int)
    for i, s in enumerate(rx):
        detected_bits[2*i] = 0 if s.real > 0 else 1
        detected_bits[2*i + 1] = 0 if s.imag > 0 else 1
    
    errors = np.sum(bits != detected_bits)
    return errors / num_bits

# Run QPSK simulation
ber_qpsk_sim = [simulate_qpsk(eb_n0, 200000) for eb_n0 in eb_n0_test]
print("QPSK simulation complete")

In [ ]:
# Compare BPSK and QPSK
plt.figure(figsize=(10, 6))

# Theoretical
plt.semilogy(eb_n0_fine, ber_bpsk_theory(eb_n0_fine), 'b-', label='BPSK Theory')
plt.semilogy(eb_n0_fine, ber_qpsk_theory(eb_n0_fine), 'g--', label='QPSK Theory')

# Simulated
ber_bpsk_arr = np.array(ber_simulated)
ber_qpsk_arr = np.array(ber_qpsk_sim)
ber_bpsk_arr[ber_bpsk_arr == 0] = 1e-7
ber_qpsk_arr[ber_qpsk_arr == 0] = 1e-7

plt.semilogy(eb_n0_test, ber_bpsk_arr, 'bo', markersize=8, label='BPSK Sim')
plt.semilogy(eb_n0_test, ber_qpsk_arr, 'gs', markersize=8, label='QPSK Sim')

plt.xlabel('Eb/N0 (dB)')
plt.ylabel('Bit Error Rate')
plt.title('BPSK vs QPSK BER Comparison')
plt.legend()
plt.grid(True, which='both')
plt.ylim(1e-6, 1)
plt.show()

print("\nNote: QPSK has same BER as BPSK at same Eb/N0, but 2x data rate!")

## Confidence Intervals

For BER simulation, the number of errors follows a binomial distribution.

Rule of thumb: Need ~100 errors for 10% relative accuracy.

In [ ]:
def bits_needed(target_ber, confidence_errors=100):
    """Calculate bits needed for reliable BER measurement."""
    return int(confidence_errors / target_ber)

print("Bits needed for reliable measurement:")
for ber in [1e-3, 1e-4, 1e-5, 1e-6]:
    print(f"  BER = {ber:.0e}: {bits_needed(ber):,} bits")

## Exercises

1. **8-PSK simulation**: Implement a simulator for 8-PSK and compare to theory.

2. **Soft vs hard decision**: Implement soft-decision decoding for BPSK. How much does it help?

3. **Fading channel**: Modify the simulator to include Rayleigh fading. How does the BER curve change?

In [ ]:
# Your code here
